## 1 Create a Database

We are going to create a database with with three datasets to efficiently access information

To do this, we will need the following imports

In [3]:
import pandas as pd
import numpy as np
import sqlite3

### Temperatures table

The temperatures dataset includes the temperatures for each month of a year for thousands of stations. There is separate file for each decade worth of years, so we add that decade to the url. Let's look at the data table for the most recent decade.

In [3]:
interval = "2011-2020"
temps_url = f"https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/decades/{interval}.csv"
temperatures = pd.read_csv(temps_url)
temperatures.head(3)

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,ACW00011604,2011,-83.0,-132.0,278.0,1040.0,1213.0,1663.0,1875.0,1723.0,1466.0,987.0,721.0,428.0
1,ACW00011604,2012,121.0,-98.0,592.0,646.0,1365.0,1426.0,1771.0,1748.0,1362.0,826.0,620.0,-234.0
2,ACW00011604,2013,-104.0,-93.0,-48.0,595.0,NaN,1612.0,1855.0,1802.0,1359.0,1042.0,601.0,NaN


The way this data is set up is not tidy, since there are 12 temperatures values in each observation. To make it tidy, we want to have one row for each month. The temperature values are also in hundreths of a degree, so we want to divide them by degree. Let's make a function to do this.

In [23]:
def prep_temp_df(df):
    """
    This function tidys and cleans the data.
    It makes each observation be the temperature for a station, year, and month
    and neatens the new columns of month and temp
    
    Input: df - a dataframe of temperatures
    
    Outputs a restructured dataframe of the temperatures
    """
    #We want to keep ID and Year as columns in the transformed table
    df = df.set_index(keys=["ID", "Year"])
    
    #Convert table
    df = df.stack()
    df = df.reset_index()
    
    #Make the new columns for Month and Temp look nice
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)

Now we want to create a database.

In [56]:
conn = sqlite3.connect("climate.db")

Now we want to add the data, one decade file at a time. So for each decade, we will read in data from the url, prepare it, and add it to the temperatures table.

In [57]:
#List with the start of each decade
decades = np.arange(1901, 2021, 10)

for start in decades:
    interval = str(start) + "-" + str(start+9)
    temps_url = f"https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/decades/{interval}.csv"
    
    #Create iterator to reach in chunk
    df_iter = pd.read_csv(temps_url, chunksize = 100000)
    
    #Iterate through the file, adding chunks
    for df in df_iter:
        cleaned = prep_temp_df(df)
        cleaned.to_sql("temperatures", conn, if_exists = "append", index = False)

### Stations table

Now we will get our next table from its url.

In [10]:
stations_url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(stations_url)
stations.head(3)

,ID,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,57.7667,11.8667,18.0,SAVE
1,AE000041196,25.3330,55.5170,34.0,SHARJAH_INTER_AIRP
2,AEM00041184,25.6170,55.9330,31.0,RAS_AL_KHAIMAH_INTE


We will want to use this table to match countries. It turns out the first two characters of the ID match the FIPS 10-4 country code, so let's add that to our table

In [11]:
stations["FIPS"] = stations["ID"].str[0:2]

Now we can add this table directly to the database

In [58]:
stations.to_sql("stations", conn, if_exists = "replace", index = False)

### Countries table

Lastly we are going to add our countries table. We can get it from the url below.

In [14]:
countries_url = "https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv"
countries = pd.read_csv(countries_url)
countries.head(3)

,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania


We are going to run into problems with SQL if the column names have spaces, so let's shorten those. It might also make sense to change Name to Country.

In [15]:
countries = countries.rename(columns = {"FIPS 10-4":"FIPS", "ISO 3166":"ISO", "Name":"Country"})
countries.head(3)

,FIPS,ISO,Country
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania


Then we can add this table to our database

In [59]:
countries.to_sql("countries", conn, if_exists = "replace", index = False)

Since we are done adding to our database, we need to close the connection.

In [60]:
conn.close()

## 2 Write a Query Function

Now we are going to write a function that will create table from the database using SQL

We want to look at temperatures for a given countries for a given month in a range of years. We want the dataframe to have columns for the station name, latititude, longitude, and country, as well as year, month, and temperature of the reading.   

To get this we will make a command. The command will select all of the columns from the abreviated name of table they came from. It will join the temperatures table with station, on id, and countries, on FIPS in order to do this. In the WHERE statement, we will include the conditions.

In [4]:
def query_climate_database(country, year_begin, year_end, month):
    """
    This function creates a dataframe of temperatures for stations in a country
    for a given month and years from the climate database
    
    Inputs:
    country - the country to look in
    year_begin - the first year to gather data for
    year_end - the last year to gather data for
    month - the month of tempterature to look at
    
    Outputs a dataframe according to the specifications
    """
    
    #The command pulls data from all three tables by joining them
    cmd = """
    SELECT S.name, S.latitude, S.longitude, C.country, T.year, T.month, T.temp
    FROM temperatures T
    LEFT JOIN stations S ON T.id = S.id
    LEFT JOIN countries C ON C.FIPS = S.FIPS
    WHERE C.country= \"""" + str(country) + """\"
    AND T.year >=""" + str(year_begin) + """
    AND T.year <=""" + str(year_end) + """
    AND T.month =""" + str(month)
    
    #Open the connection to execute the command
    conn = sqlite3.connect("climate.db")
    df = pd.read_sql_query(cmd, conn)
    conn.close()
    return df

Lets do an example where we look at temperatures in August for stations in Germany from 2009 to 2012.

In [3]:
query_climate_database("Germany", 2009, 2012, 8).head(3)

,NAME,LATITUDE,LONGITUDE,Country,Year,Month,Temp
0,BREMEN,53.0464,8.7992,Germany,2009,8,18.46
1,BREMEN,53.0464,8.7992,Germany,2010,8,17.14
2,TRIER,49.7517,6.6467,Germany,2009,8,19.53


## 3 Write a Geographic Scatter Function for Yearly Temperature Increases

Our next task is to create a function our query that will address the following question:

*How does the average yearly change in temperature vary within a given country?*

To do this we are going to group the data by station and find the change using linear regression. For this we will need a function to use in the apply.

In [5]:
from sklearn.linear_model import LinearRegression

def coef(data_group):
    """
    This function finds the linear regression equation for a data group
    
    Input: data_group - as data group from the groupby function
    
    Output the rounded first coefficient of the equation
    """
    x = data_group[["Year"]]
    y = data_group["Temp"]
    LR = LinearRegression()
    LR.fit(x,y)
    return round(LR.coef_[0],3)

We will do this with plotly.

In [6]:
from plotly import express as px

Make function

In [7]:
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    df = query_climate_database(country, year_begin, year_end, month)
    obs = df.groupby(["NAME"])["Month"].transform(np.sum) / month
    df = df[obs >= min_obs]
    df = df.reset_index()
    coefs = df.groupby(["NAME", "LATITUDE", "LONGITUDE"]).apply(coef).reset_index()
    coefs["Yearly\nIncrease"] = coefs[0]
    title = "Yearly Temperature Increase in Month " + str(month)
    title += "for stations in " + country +" "+ str(year_begin)
    title += "-" + str(year_end)
    fig = px.scatter_mapbox(coefs,
                            lat = "LATITUDE",
                            lon = "LONGITUDE",
                            color = "Yearly\nIncrease",
                            color_continuous_midpoint = 0,
                            hover_name = "NAME",
                            title = title,
                            mapbox_style = "carto-positron",
                            **kwargs)
    return fig

In [8]:
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig = temperature_coefficient_plot("India", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   #mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

In [9]:
from plotly.io import write_html
write_html(fig, "plot3.html")